# Bouncing Balls

`Goal is`: Compare the average speed of each ball

---

In [50]:
import moviepy.video.io.VideoFileClip as mpy
import moviepy.editor as mpyEditor
import numpy as np
import cv2
import winsound
import gc

In [51]:
def showImage(image):
    cv2.imshow('Resized Window', image)  #Display image
    cv2.waitKey(0)  #Wait for keyboard button press
    cv2.destroyAllWindows()  #Exit window and destroy all windows using

def showImageResized(image, width, height):
    cv2.namedWindow('Resized Window', cv2.WINDOW_NORMAL)  #Create window
    cv2.resizeWindow('Resized Window', width, height)  #Resize the window
    cv2.imshow('Resized Window', image)  #Display image
    cv2.waitKey(0)  #Wait for keyboard button press
    cv2.destroyAllWindows()  #Exit window and destroy all windows using

## Red Ball

In [52]:
biped_vid = mpy.VideoFileClip("movie_001.avi")

frame_count = biped_vid.reader.nframes#about 150
video_fps = biped_vid.fps

image_list = []
image_list_red = []
V_list_red = []

color = (245, 135, 66) 
counter = 0

#p0 = np.array([[[380., 320.]]]).astype(np.float32)
p0 = np.array([[[399., 324.]]]).astype(np.float32)

lk_params = dict( winSize  = (51,51),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))


for i in range(frame_count-1):
    current_frame = biped_vid.get_frame(i*1.0/video_fps) #To get frames by ID 
    next_frame    = biped_vid.get_frame((i+1)*1.0/video_fps)
     
    #showImage(cv2.cvtColor(current_frame, cv2.COLOR_RGB2BGR))
    #break
    ##########RGB to GRAY###########
    
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_RGB2GRAY)
    next_frame_gray = cv2.cvtColor(next_frame, cv2.COLOR_RGB2GRAY)
    
    #background subtraction
    
    
    current_frame_g_channel = current_frame[:, :, 1] # Green channel of the image
    current_frame_r_channel = current_frame[:, :, 0] # Red channel of the image
    
    next_frame_g_channel    = next_frame[:, :, 1] # Green channel of the image
    next_frame_r_channel    = next_frame[:, :, 0]
    
    foreground_current_frame = np.logical_and(current_frame_g_channel < 65, current_frame_r_channel > 120)
    foreground_next_frame    = np.logical_and(next_frame_g_channel < 65, next_frame_r_channel > 120)
    
    nonzero_y_current_frame, nonzero_x_current_frame = np.nonzero(foreground_current_frame) #Where are the true values
    nonzero_y_next_frame, nonzero_x_next_frame       = np.nonzero(foreground_next_frame) #Where are the true values
    
    copy_current_frame_gray = current_frame_gray.copy()
    copy_next_frame_gray    = next_frame_gray.copy()
    
    copy_current_frame_gray[nonzero_y_current_frame, nonzero_x_current_frame] = 0 #makes the color of hand black
    copy_next_frame_gray[nonzero_y_next_frame, nonzero_x_next_frame] = 0  #makes the color of hand black
    
    current_frame_gray = current_frame_gray - copy_current_frame_gray
    next_frame_gray    = next_frame_gray - copy_next_frame_gray
    
    #showImage(current_frame_gray)
    #break
    
    #print(p0)
    
    #apply lucas kanade algorithm
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(current_frame_gray, next_frame_gray, p0, None, **lk_params)
    
    ###Check Borders ###
    
    if p1[0,0,1] < 0:
        p1[0,0,1] = np.float32(0.)
    if p1[0,0,1] > 479:
        p1[0,0,1] = np.float32(479.)
    if p1[0,0,0] < 0:
        p1[0,0,0] = np.float32(0.)
    if p1[0,0,0] > 719:
        p1[0,0,0] = np.float32(719.)
        
    ###Get Vx and Vy###
    
    V = np.sqrt(np.power((p1[0,0,0] - p0[0,0,0]), 2) + np.power((p1[0,0,1] - p0[0,0,1]), 2))
    #Vy = np.power((p1[0,0,1] - p0[0,0,1]), 2)
    
    #Vx_list_red.append(Vx)
    #Vy_list_red.append(Vy)
    
    V_list_red.append(V)
    
    #print("st: ", st)
    #print("p0: ", p0)
    #print("p1: ", p1)
    
    # Select good points
    good_new = p1[st==1]
    if not good_new.any():
        counter += 1
        good_new = p1.reshape(1,2)
    #print("good new: ", good_new)
    good_old = p0[st==1]
    if not good_old.any():
        good_old = p0.reshape(1,2)
    #print("good old: ", good_old)
    # draw the tracks
    for t,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()

        u = c + (a - c)*3
        u = np.float32(u)
        v = d + (b - d)*3
        v = np.float32(v)
        
        sol_ust = (c - np.float32(25), d - np.float32(25))
        sag_alt = (c + np.float32(25), d + np.float32(25))
        
        #print("sol ust: ", sol_ust, "sag ust: ", sag_alt)
        
        #break
        #mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        #frame = cv.circle(frame,(a,b),5,color[i].tolist(),3)
        cv2.arrowedLine(current_frame, (c,d), (u,v), (0, 0, 255), 2)
        #cv2.circle(current_frame, (c,d), 5, (0,255,0), 1)
        #cv2.rectangle(current_frame, sol_ust, sag_alt, (0, 0, 255), 3)
    
    p0 = good_new.reshape(-1,1,2)
        
    #image_list_red.append(current_frame)
    image_list.append(current_frame)
    
    #print the work done up to now
    if i % (frame_count//10) == 0: 
        print("processing\t", round((i/frame_count)*100), "%")

#create an alert when the function is done
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)


processing	 0 %
processing	 10 %
processing	 20 %
processing	 30 %
processing	 40 %
processing	 50 %
processing	 60 %
processing	 70 %
processing	 80 %
processing	 90 %
processing	 100 %


**create video**

In [53]:
counter # points that LK couldnt catch

804

In [54]:
np.average(V_list_red) ## 10.69 pixel per frame

13.421680799307449

In [55]:
#clip = mpyEditor.ImageSequenceClip(image_list, fps = video_fps)
#clip.write_videofile('part3_red_ball.mp4', codec='libx264')

## Blue Ball

In [56]:
biped_vid = mpy.VideoFileClip("movie_001.avi")

frame_count = biped_vid.reader.nframes#about 150
video_fps = biped_vid.fps

image_list_blue = []
V_list_blue = []

color = (245, 135, 66) 
counter = 0

#p0 = np.array([[[380., 320.]]]).astype(np.float32)
p0 = np.array([[[207., 176.]]]).astype(np.float32)

lk_params = dict( winSize  = (51,51),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

for i in range(frame_count-1):
    current_frame = biped_vid.get_frame(i*1.0/video_fps) #To get frames by ID 
    next_frame    = biped_vid.get_frame((i+1)*1.0/video_fps)
     
    #showImage(cv2.cvtColor(current_frame, cv2.COLOR_RGB2BGR))
    #break
    ##########RGB to GRAY###########
    
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_RGB2GRAY)
    next_frame_gray = cv2.cvtColor(next_frame, cv2.COLOR_RGB2GRAY)
    
    #background subtraction
    
    
    current_frame_b_channel = current_frame[:, :, 2] # Blue channel of the image
    current_frame_r_channel = current_frame[:, :, 0] # Red channel of the image
    
    next_frame_b_channel    = next_frame[:, :, 2] # Blue channel of the image
    next_frame_r_channel    = next_frame[:, :, 0] # Red channel of the image
    
    foreground_current_frame = np.logical_and(current_frame_b_channel > 150, current_frame_r_channel < 100)
    foreground_next_frame    = np.logical_and(next_frame_b_channel > 150, next_frame_r_channel < 100)
    
    nonzero_y_current_frame, nonzero_x_current_frame = np.nonzero(foreground_current_frame) #Where are the true values
    nonzero_y_next_frame, nonzero_x_next_frame       = np.nonzero(foreground_next_frame) #Where are the true values
    
    copy_current_frame_gray = current_frame_gray.copy()
    copy_next_frame_gray    = next_frame_gray.copy()
    
    copy_current_frame_gray[nonzero_y_current_frame, nonzero_x_current_frame] = 0 #makes the color of hand black
    copy_next_frame_gray[nonzero_y_next_frame, nonzero_x_next_frame] = 0  #makes the color of hand black
    
    current_frame_gray = current_frame_gray - copy_current_frame_gray
    next_frame_gray    = next_frame_gray - copy_next_frame_gray
    
    #showImage(current_frame_gray)
    #break
    
    #print(p0)
    
    #apply lucas kanade algorithm
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(current_frame_gray, next_frame_gray, p0, None, **lk_params)
    
    ### Check Borders ###
    
    if p1[0,0,1] < 0:
        p1[0,0,1] = np.float32(0.)
    if p1[0,0,1] > 479:
        p1[0,0,1] = np.float32(479.)
    if p1[0,0,0] < 0:
        p1[0,0,0] = np.float32(0.)
    if p1[0,0,0] > 719:
        p1[0,0,0] = np.float32(719.)
    
    ###Get Vx and Vy###
    
    V = np.sqrt(np.power((p1[0,0,0] - p0[0,0,0]), 2) + np.power((p1[0,0,1] - p0[0,0,1]), 2))
    #Vy = np.power((p1[0,0,1] - p0[0,0,1]), 2)
    
    #Vx_list_blue.append(Vx)
    #Vy_list_blue.append(Vy)
    
    V_list_blue.append(V)
    
    #print("st: ", st)
    #print("p0: ", p0)
    #print("p1: ", p1)
    
    # Select good points
    good_new = p1[st==1]
    if not good_new.any():
        counter += 1
        good_new = p1.reshape(1,2)
    #print("good new: ", good_new)
    good_old = p0[st==1]
    if not good_old.any():
        good_old = p0.reshape(1,2)
    #print("good old: ", good_old)
    # draw the tracks
    for t,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()

        u = c + (a - c)*3
        u = np.float32(u)
        v = d + (b - d)*3
        v = np.float32(v)
        
        sol_ust = (c - np.float32(25), d - np.float32(25))
        sag_alt = (c + np.float32(25), d + np.float32(25))
        
        #print("sol ust: ", sol_ust, "sag ust: ", sag_alt)
        
        #break
        #mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        #frame = cv.circle(frame,(a,b),5,color[i].tolist(),3)
        cv2.arrowedLine(image_list[i], (c,d), (u,v), (0, 255, 0), 2)
        #cv2.circle(current_frame, (c,d), 5, (0,255,0), 2)
        #cv2.rectangle(current_frame, sol_ust, sag_alt, (255, 0, 0), 3)
    
    p0 = good_new.reshape(-1,1,2)
        
    #image_list_blue.append(current_frame)
    
    #print the work done up to now
    if i % (frame_count//10) == 0: 
        print("processing\t", round((i/frame_count)*100), "%")

#create an alert when the function is done
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)


processing	 0 %
processing	 10 %
processing	 20 %
processing	 30 %
processing	 40 %
processing	 50 %
processing	 60 %
processing	 70 %
processing	 80 %
processing	 90 %
processing	 100 %


In [57]:
np.average(V_list_blue) ## 10.69 pixel per frame

13.411067280260966

In [58]:
counter

466

**create video**

In [59]:
#clip = mpyEditor.ImageSequenceClip(image_list, fps = video_fps) # image list blue
#clip.write_videofile('part3_blue_ball.mp4', codec='libx264')

## Green Ball

In [60]:
biped_vid = mpy.VideoFileClip("movie_001.avi")

frame_count = biped_vid.reader.nframes#about 150
video_fps = biped_vid.fps

image_list_green = []
V_list_green = []

color = (245, 135, 66) 
counter = 0

#p0 = np.array([[[380., 320.]]]).astype(np.float32)
p0 = np.array([[[333., 180.]]]).astype(np.float32)

lk_params = dict( winSize  = (51,51),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

for i in range(frame_count-1):
    current_frame = biped_vid.get_frame(i*1.0/video_fps) #To get frames by ID 
    next_frame    = biped_vid.get_frame((i+1)*1.0/video_fps)
     
    #showImage(cv2.cvtColor(current_frame, cv2.COLOR_RGB2BGR))
    #break
    ##########RGB to GRAY###########
    
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_RGB2GRAY)
    next_frame_gray = cv2.cvtColor(next_frame, cv2.COLOR_RGB2GRAY)
    
    #background subtraction
    
    
    current_frame_g_channel = current_frame[:, :, 1] # Blue channel of the image
    current_frame_r_channel = current_frame[:, :, 0] # Red channel of the image
    
    next_frame_g_channel    = next_frame[:, :, 1] # Blue channel of the image
    next_frame_r_channel    = next_frame[:, :, 0] # Red channel of the image
    
    foreground_current_frame = np.logical_and(current_frame_g_channel > 150, current_frame_r_channel < 100)
    foreground_next_frame    = np.logical_and(next_frame_g_channel > 150, next_frame_r_channel < 100)
    
    nonzero_y_current_frame, nonzero_x_current_frame = np.nonzero(foreground_current_frame) #Where are the true values
    nonzero_y_next_frame, nonzero_x_next_frame       = np.nonzero(foreground_next_frame) #Where are the true values
    
    copy_current_frame_gray = current_frame_gray.copy()
    copy_next_frame_gray    = next_frame_gray.copy()
    
    copy_current_frame_gray[nonzero_y_current_frame, nonzero_x_current_frame] = 0 #makes the color of hand black
    copy_next_frame_gray[nonzero_y_next_frame, nonzero_x_next_frame] = 0  #makes the color of hand black
    
    current_frame_gray = current_frame_gray - copy_current_frame_gray
    next_frame_gray    = next_frame_gray - copy_next_frame_gray
    
    #showImage(current_frame_gray)
    #break
    
    #print(p0)
    
    #apply lucas kanade algorithm
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(current_frame_gray, next_frame_gray, p0, None, **lk_params)
    
    ###Check Borders ###
    
    if p1[0,0,1] < 0:
        p1[0,0,1] = np.float32(0.)
    if p1[0,0,1] > 479:
        p1[0,0,1] = np.float32(479.)
    if p1[0,0,0] < 0:
        p1[0,0,0] = np.float32(0.)
    if p1[0,0,0] > 719:
        p1[0,0,0] = np.float32(719.)
    
    ###Get Vx and Vy###
    
    V = np.sqrt(np.power((p1[0,0,0] - p0[0,0,0]), 2) + np.power((p1[0,0,1] - p0[0,0,1]), 2))
    #Vy = np.power((p1[0,0,1] - p0[0,0,1]), 2)
    
    #Vx_list_green.append(Vx)
    #Vy_list_green.append(Vy)
    
    V_list_green.append(V)
    
    #print("st: ", st)
    #print("p0: ", p0)
    #print("p1: ", p1)
    
    # Select good points
    good_new = p1[st==1]
    if not good_new.any():
        counter += 1
        good_new = p1.reshape(1,2)
    #print("good new: ", good_new)
    good_old = p0[st==1]
    if not good_old.any():
        good_old = p0.reshape(1,2)
    #print("good old: ", good_old)
    # draw the tracks
    for t,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()

        u = c + (a - c)*3
        u = np.float32(u)
        v = d + (b - d)*3
        v = np.float32(v)
        
        sol_ust = (c - np.float32(25), d - np.float32(25))
        sag_alt = (c + np.float32(25), d + np.float32(25))
        
        #print("sol ust: ", sol_ust, "sag ust: ", sag_alt)
        
        #break
        #mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        #frame = cv.circle(frame,(a,b),5,color[i].tolist(),3)
        cv2.arrowedLine(image_list[i], (c,d), (u,v), (255, 0, 0), 2)
        #cv2.circle(current_frame, (c,d), 5, (0,255,0), 1)
        #cv2.rectangle(current_frame, sol_ust, sag_alt, (255, 0, 0), 3)
    
    p0 = good_new.reshape(-1,1,2)
        
    #image_list_green.append(current_frame)
    
    #print the work done up to now
    if i % (frame_count//10) == 0: 
        print("processing\t", round((i/frame_count)*100), "%")

#create an alert when the function is done
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)
gc.collect()


processing	 0 %
processing	 10 %
processing	 20 %
processing	 30 %
processing	 40 %
processing	 50 %
processing	 60 %
processing	 70 %
processing	 80 %
processing	 90 %
processing	 100 %


9212

In [61]:
np.average(V_list_green) ## 10.53 pixel per frame

14.691866520738518

In [62]:
counter

1070

In [63]:
#clip = mpyEditor.ImageSequenceClip(image_list, fps = video_fps)
#clip.write_videofile('part3_green_ball.mp4', codec='libx264')

## Pink Ball

In [64]:
biped_vid = mpy.VideoFileClip("movie_001.avi")

frame_count = biped_vid.reader.nframes#about 150
video_fps = biped_vid.fps

image_list_pink = []
V_list_pink = []

color = (245, 135, 66) 
counter = 0

#p0 = np.array([[[380., 320.]]]).astype(np.float32)
p0 = np.array([[[484., 93.]]]).astype(np.float32)

lk_params = dict( winSize  = (51,51),
                  maxLevel = 3,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

for i in range(frame_count-1):
    current_frame = biped_vid.get_frame(i*1.0/video_fps) #To get frames by ID 
    next_frame    = biped_vid.get_frame((i+1)*1.0/video_fps)
     
    #showImage(cv2.cvtColor(current_frame, cv2.COLOR_RGB2BGR))
    #break
    ##########RGB to GRAY###########
    
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_RGB2GRAY)
    next_frame_gray = cv2.cvtColor(next_frame, cv2.COLOR_RGB2GRAY)
    
    #background subtraction
    
    
    current_frame_b_channel = current_frame[:, :, 2] # Blue channel of the image
    current_frame_g_channel = current_frame[:, :, 1] # Green channel of the image
    current_frame_r_channel = current_frame[:, :, 0] # Red channel of the image
    
    next_frame_b_channel    = next_frame[:, :, 2] # Blue channel of the image
    next_frame_g_channel    = next_frame[:, :, 1] # green channel of the image
    next_frame_r_channel    = next_frame[:, :, 0] # Red channel of the image
    
    foreground_current_frame = np.logical_and(current_frame_b_channel > 225, current_frame_r_channel > 225)
    foreground_current_frame = np.logical_and(foreground_current_frame, current_frame_g_channel < 225)
    
    foreground_next_frame    = np.logical_and(next_frame_b_channel > 225, next_frame_r_channel < 225)
    foreground_next_frame    = np.logical_and(foreground_next_frame, next_frame_g_channel < 225)
    
    nonzero_y_current_frame, nonzero_x_current_frame = np.nonzero(foreground_current_frame) #Where are the true values
    nonzero_y_next_frame, nonzero_x_next_frame       = np.nonzero(foreground_next_frame) #Where are the true values
    
    copy_current_frame_gray = current_frame_gray.copy()
    copy_next_frame_gray    = next_frame_gray.copy()
    
    copy_current_frame_gray[nonzero_y_current_frame, nonzero_x_current_frame] = 50 #makes the color of hand black
    copy_next_frame_gray[nonzero_y_next_frame, nonzero_x_next_frame] = 50  #makes the color of hand black
    
    current_frame_gray = current_frame_gray - copy_current_frame_gray
    next_frame_gray    = next_frame_gray - copy_next_frame_gray
    
    
    
    #showImage(current_frame_gray)
    
    
    #print(p0)
    
    #apply lucas kanade algorithm
    
    p1, st, err = cv2.calcOpticalFlowPyrLK(current_frame_gray, next_frame_gray, p0, None, **lk_params)
    
    ###Check Borders ###
    
    if p1[0,0,1] < 0:
        p1[0,0,1] = np.float32(0.)
    if p1[0,0,1] > 479:
        p1[0,0,1] = np.float32(479.)
    if p1[0,0,0] < 0:
        p1[0,0,0] = np.float32(0.)
    if p1[0,0,0] > 719:
        p1[0,0,0] = np.float32(719.)
    
    ###Get Vx and Vy###
    
    V = np.sqrt(np.power((p1[0,0,0] - p0[0,0,0]), 2) + np.power((p1[0,0,1] - p0[0,0,1]), 2))
    #Vy = np.power((p1[0,0,1] - p0[0,0,1]), 2)
    
    #Vx_list_pink.append(Vx)
    #Vy_list_pink.append(Vy)
    
    V_list_pink.append(V)
    
    #print("st: ", st)
    #print("p0: ", p0)
    #print("p1: ", p1)
    
    # Select good points
    good_new = p1[st==1]
    if not good_new.any():
        counter += 1
        good_new = p1.reshape(1,2)
    #print("good new: ", good_new)
    good_old = p0[st==1]
    if not good_old.any():
        good_old = p0.reshape(1,2)
    #print("good old: ", good_old)
    # draw the tracks
    for t,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()

        u = c + (a - c)*3
        u = np.float32(u)
        v = d + (b - d)*3
        v = np.float32(v)
        
        sol_ust = (c - np.float32(25), d - np.float32(25))
        sag_alt = (c + np.float32(25), d + np.float32(25))
        
        #print("sol ust: ", sol_ust, "sag ust: ", sag_alt)
        
        #break
        #mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        #frame = cv.circle(frame,(a,b),5,color[i].tolist(),3)
        cv2.arrowedLine(image_list[i], (c,d), (u,v), (252, 186, 3), 2)
        #cv2.circle(current_frame, (c,d), 5, (0,255,0), 1)
        #cv2.rectangle(current_frame, sol_ust, sag_alt, (252, 186, 3), 3)
    
    p0 = good_new.reshape(-1,1,2)
        
    #image_list_pink.append(current_frame)
    
    #print the work done up to now
    if i % (frame_count//10) == 0: 
        print("processing\t", round((i/frame_count)*100), "%")

#create an alert when the function is done
duration = 1000  # milliseconds
freq = 440  # Hz
winsound.Beep(freq, duration)


processing	 0 %
processing	 10 %
processing	 20 %
processing	 30 %
processing	 40 %
processing	 50 %
processing	 60 %
processing	 70 %
processing	 80 %
processing	 90 %
processing	 100 %


In [65]:
np.average(V_list_pink) ## 4.12 pixel per frame

14.08482568942528

In [66]:
counter

415

In [67]:
clip = mpyEditor.ImageSequenceClip(image_list, fps = video_fps)
clip.write_videofile('part3_results.mp4', codec='libx264')

t:   3%|██▏                                                               | 72/2165 [00:00<00:07, 296.11it/s, now=None]

Moviepy - Building video part3_results.mp4.
Moviepy - Writing video part3_results.mp4



Moviepy - Done !
Moviepy - video ready part3_results.mp4
